In [ ]:
!pip install datasets
!pip install transformers
!pip install bitsandbytes
from datasets import load_dataset
import pandas as pd
import json
import os
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm

In [ ]:
!pip install together
from together import Together
os.environ["TOGETHER_API_KEY"] = "65abe69039d40b30b01f8c4fbcd0c48be19434e153bd14929f98332fdf8b855c"
client = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 1.1 MB/s eta 0:00:00


In [16]:
def run_inference_zero_shot(model, question, choices):
    prompt = f"Choose the answer to the given question from below options.\n\nQuestion: {question}\nOptions:\nA. {choices[0]}\nB. {choices[1]}\nC. {choices[2]}\nD. {choices[3]}"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "assistant", "content": "You are an expert assistant in mathematics who can solve all the questions correctly."},
            {"role": "user", "content": prompt}
        ]
    )

    response_text = response.choices[0].message.content.strip()
    return response_text

In [17]:
def run_inference_cot(model, question, choices):
    prompt = f"Choose the answer to the given question from below options. Think step by step.\n\nQuestion: {question}\nOptions:\nA. {choices[0]}\nB. {choices[1]}\nC. {choices[2]}\nD. {choices[3]}"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "assistant", "content": "You are an expert assistant in mathematics who can solve all the questions correctly."},
            {"role": "user", "content": prompt}
        ]
    )

    response_text = response.choices[0].message.content.strip()
    return response_text

In [18]:
def run_inference_react(model, question, choices):
    prompt = f"I need your help and I want your assistance in solving mathematical problems. When presented with a mathematical problems, you need to do the following steps: Analyze the Problem: Understand the question and determine the best approach to solve it. Decide on Action: Choose a specific action to take, such as performing a calculation or applying a relevant formula. Note the Result: Record the outcome of your action. Continue to repeat steps 1-3 as necessary until you arrive at the final answer. Please solve the following question and select the correct answer. Provide reasoning as you progress.\n\nQuestion: {question}\nOptions:\nA. {choices[0]}\nB. {choices[1]}\nC. {choices[2]}\nD. {choices[3]}"

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "assistant", "content": "You are an expert assistant in mathematics who can solve all the questions correctly."},
            {"role": "user", "content": prompt}
        ]
    )

    response_text = response.choices[0].message.content.strip()
    return response_text

In [19]:
def evaluate_responses(model_name, prompt_type, output_file):
    with open('/content/test_mmlu_fixed.json', 'r') as file:
        data = json.load(file)
    # data = data[:len(data) // 2]
    save_results = []

    for idx, item in enumerate(tqdm(data, desc=f"Evaluating {model_name} with {prompt_type}")):
        question = item['question']
        choices = item['choices']

        if prompt_type == "zero_shot":
            response = run_inference_zero_shot(model_name, question, choices)
        elif prompt_type == "cot":
            response = run_inference_cot(model_name, question, choices)
        elif prompt_type == "react":
            response = run_inference_react(model_name, question, choices)

        print(f"Model response: {response}")
        item['response'] = response
        save_results.append(item)

        if (idx + 1) % 10 == 0:
            existing_results = []
            if os.path.exists(output_file):
                with open(output_file, 'r') as file:
                    existing_results = json.load(file)
            existing_results.extend(save_results)
            with open(output_file, 'w') as file:
                json.dump(existing_results, file, indent=4)
            save_results = []

    if save_results:
        existing_results = []
        if os.path.exists(output_file):
            with open(output_file, 'r') as file:
                existing_results = json.load(file)
        existing_results.extend(save_results)
        with open(output_file, 'w') as file:
            json.dump(existing_results, file, indent=4)


###Gemma

In [21]:
# (Zero Shot)
evaluate_responses("google/gemma-2b-it", "zero_shot", "gemma_zero_shot.json")

# (Chain of Thought)
evaluate_responses("google/gemma-2b-it", "cot", "gemma_cot.json")

# (ReAct)
evaluate_responses("google/gemma-2b-it", "react", "gemma_react.json")

Evaluating google/gemma-2b-it with zero_shot:   1%|          | 1/100 [00:01<01:49,  1.11s/it]

Model response: E. k = 1 and n = 0

Answer: A. k = 0 and n = 1

Explanation:

For any real value of x in the interval [0, 1], the equation e^x + x - 2 = 0 has either one real solution (if x = log(2)) or no real solutions.

Therefore, k = 0 and n = 1.


Evaluating google/gemma-2b-it with zero_shot:   2%|▏         | 2/100 [00:02<02:17,  1.40s/it]

Model response: Answer: C. 2

Explanation: An abelian group G of order 16 is isomorphic to a direct sum of two cyclic groups of order 4. A direct sum of two cyclic groups of order k is a group of order mk, where m and k are relatively prime.

In this case, the two cyclic groups of order 4 are generated by the elements x and x^2. The element x + x + x + x = 0 is not in the span of x and x^2, so it is not an element of the direct sum.

Therefore, there are two additive abelian groups of order 16 that have the property that x + x + x + x = 0 for each x in G.


Evaluating google/gemma-2b-it with zero_shot:   3%|▎         | 3/100 [00:05<03:22,  2.09s/it]

...

Model response: Answer: A. x^2 + y^2 = 1

**Reasoning:**

**1. Analyze the Problem:**

* The equation x^2 = y + 4 represents a parabola.
* The question asks for the circle with the greatest number of points of intersection with the parabola.
* A circle is defined by its radius, which is half the distance from the center to any point on the circle.

**2. Decide on Action:**

* We need to find the radius of the circle that has the greatest number of points of intersection with the parabola.

**3. Note the Result:**

* The radius of the circle with the greatest number of points of intersection is half the distance between the vertex of the parabola (0, 4) and the center of the circle.
* The center of the circle is located at (0, -4).
* The radius is half the distance between (0, 4) and (0, -4), which is 4.

**Therefore, the radius of the circle with the greatest number of points of intersection with the parabola x^2 = y + 4 is 4.**


Evaluating google/gemma-2b-it with react:  99%|█████████▉| 99/100 [04:13<00:02,  2.85s/it]

Model response: Correct Answer: A. 24

**Reasoning:**

**Step 1: Analyze the Problem**

* The problem is asking for the number of subsets of X that contain exactly 4 elements.
* We are given that the number of subsets of X that contain exactly 2 elements is 14 more than the number of subsets of X that contain exactly 3 elements.
* We need to find the number of subsets of X that contain exactly 4 elements.

**Step 2: Decide on an Action**

* We need to find the number of subsets of X that contain exactly 4 elements.

**Step 3: Note the Result**

* Let X be the set of all elements in the problem.
* We are told that the number of subsets of X that contain exactly 2 elements is 14 more than the number of subsets of X that contain exactly 3 elements.
* This means that |X - {2 elements}| = |X - {3 elements}| + 14.

**Step 4: Solve for the Number ofsubsets of X that contain exactly 4 elements**

* From the equation above, we get: |X - {2 elements}| = |X - {3 elements}| - 14.
* Since the numbe

Evaluating google/gemma-2b-it with react: 100%|██████████| 100/100 [04:14<00:00,  2.55s/it]

Model response: Solution: A. (1, 6)

**Reasoning:**

1. **Analysis:** We are given a linear transformation T: R^2 -> R^2 that maps the points (1, 2) and (-1, 2) to (2, 3) and (2, -3), respectively.
2. **Action:** We need to find the image of the point (2, 1) under the transformation.
3. **Result:** Applying the transformation to the point (2, 1), we get (1, 6).

Therefore, the correct answer is A. (1, 6).


###Llama

In [22]:
evaluate_responses("meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo", "zero_shot", "llama_zero_shot.json")
evaluate_responses("meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo", "cot", "llama_cot.json")
evaluate_responses("meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo", "react", "llama_react.json")

Evaluating meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo with zero_shot:   1%|          | 1/100 [00:03<05:08,  3.11s/it]

Model response: To solve this problem, we need to analyze the given equation e^x + x - 2 = 0.

First, let's consider the interval [0, 1]. We can check the values of the function f(x) = e^x + x - 2 at the endpoints of the interval.

At x = 0, f(0) = e^0 + 0 - 2 = 1 - 2 = -1.
At x = 1, f(1) = e^1 + 1 - 2 = e + 1 - 2 = e - 1.

Since e ≈ 2.718, we have e - 1 ≈ 1.718, which is greater than 0. This means that the function f(x) is positive at x = 1.

Now, let's consider the value of f(x) at x = 0. We have f(0) = -1, which is negative. This means that the function f(x) is negative at x = 0.

Since the function f(x) is continuous and differentiable, it must cross the x-axis at least once in the interval [0, 1]. Therefore, there is at least one real solution in the interval [0, 1].

Now, let's consider the interval (1, ∞). We can check the behavior of the function f(x) as x approaches infinity.

As x approaches infinity, e^x approaches infinity, and x approaches infinity. Therefore, the function

Evaluating meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo with zero_shot:   2%|▏         | 2/100 [00:11<09:47,  6.00s/it]

Model response: To solve this problem, we need to find the additive abelian groups of order 16 that satisfy the condition x + x + x + x = 0 for each x in G.

The additive abelian groups of order 16 are:

1. ℤ₁₆ (cyclic group of order 16)
2. ℤ₈ × ℤ₂ (direct product of ℤ₈ and ℤ₂)
3. ℤ₄ × ℤ₄ (direct product of ℤ₄ and ℤ₄)
4. ℤ₄ × ℤ₂ × ℤ₂ (direct product of ℤ₄, ℤ₂, and ℤ₂)
5. ℤ₂ × ℤ₂ × ℤ₂ × ℤ₂ (direct product of ℤ₂, ℤ₂, ℤ₂, and ℤ₂)

Now, let's check which of these groups satisfy the condition x + x + x + x = 0 for each x in G.

1. ℤ₁₆: This group does not satisfy the condition because 4 ≠ 0.
2. ℤ₈ × ℤ₂: This group does not satisfy the condition because 2 + 2 + 2 + 2 = 8 ≠ 0.
3. ℤ₄ × ℤ₄: This group satisfies the condition because 2 + 2 + 2 + 2 = 0.
4. ℤ₄ × ℤ₂ × ℤ₂: This group does not satisfy the condition because 2 + 2 + 2 + 2 = 0, but 4 + 4 + 4 + 4 = 0 as well, however, 2 + 2 + 2 + 4 = 0, but 2 + 2 + 4 + 4 = 0 as well, however, 2 + 4 + 4 + 4 = 0, but 4 + 4 + 4 + 4 = 0 as well, however, 2 +

Evaluating meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo with zero_shot:   3%|▎         | 3/100 [00:13<06:51,  4.24s/it]

...

Model response: I'll follow the steps you provided to solve the problem.

**Step 1: Analyze the Problem**

We are given a parabola equation x^2 = y + 4 and four circle equations. We need to find which circle has the greatest number of points of intersection with the parabola.

**Step 2: Decide on Action**

To find the points of intersection, we need to substitute the expression for y from the parabola equation into the circle equations. This will give us a new equation in terms of x, which we can then solve to find the x-coordinates of the points of intersection.

**Step 3: Note the Result**

Let's substitute y = x^2 - 4 into each of the circle equations:

A. x^2 + (x^2 - 4)^2 = 1
B. x^2 + (x^2 - 4)^2 = 2
C. x^2 + (x^2 - 4)^2 = 9
D. x^2 + (x^2 - 4)^2 = 16

Now, let's simplify each equation:

A. x^2 + x^4 - 8x^2 + 16 = 1
x^4 - 7x^2 + 15 = 0

B. x^2 + x^4 - 8x^2 + 16 = 2
x^4 - 7x^2 + 14 = 0

C. x^2 + x^4 - 8x^2 + 16 = 9
x^4 - 7x^2 + 7 = 0

D. x^2 + x^4 - 8x^2 + 16 = 16
x^4 - 7x^2 = 0

No

Evaluating meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo with react:  99%|█████████▉| 99/100 [06:37<00:03,  3.84s/it]

Model response: To solve this problem, we will follow the steps you provided.

**Step 1: Analyze the Problem**

We are given a finite set X with a specific property. The property states that the number of subsets of X that contain exactly 3 elements is 14 more than the number of subsets of X that contain exactly 2 elements. We need to find the number of subsets of X that contain exactly 4 elements.

**Step 2: Decide on Action**

Let's denote the total number of elements in set X as 'n'. The number of subsets of X that contain exactly 2 elements can be calculated using the combination formula: C(n, 2) = n! / (2! * (n-2)!). Similarly, the number of subsets of X that contain exactly 3 elements can be calculated using the combination formula: C(n, 3) = n! / (3! * (n-3)!).

We are given that C(n, 3) = C(n, 2) + 14. We can use this equation to find the value of 'n'.

**Step 3: Note the Result**

Using the combination formula, we can rewrite the equation as:

n! / (3! * (n-3)!) = n! / (2! * (

Evaluating meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo with react: 100%|██████████| 100/100 [06:40<00:00,  4.01s/it]

Model response: I'll follow the steps you provided to solve the problem.

**Step 1: Analyze the Problem**
We are given a linear transformation T that maps points in R^2 to R^2. We need to find the image of the point (2, 1) under T.

**Step 2: Decide on Action**
To find the image of (2, 1) under T, we can try to find the matrix representation of T. Since we know the images of two points, (1, 2) and (-1, 2), we can use this information to find the matrix representation of T.

**Step 3: Note the Result**
Let's denote the matrix representation of T as A. We know that:

T(1, 2) = (2, 3)
T(-1, 2) = (2, -3)

We can write these equations in matrix form as:

A(1, 2) = (2, 3)
A(-1, 2) = (2, -3)

We can rewrite these equations as:

A = (2, 3)
A = (2, -3)

Now, we can use these equations to find the matrix representation of T.

**Step 4: Continue to Repeat Steps 1-3**
We can rewrite the equations as:

A(1, 2) = (2, 3)
A(-1, 2) = (2, -3)

We can add the two equations to eliminate the (2, 2) term:



###Phi

In [ ]:
evaluate_responses("microsoft/Phi-3.5-mini-instruct", "zero_shot", "phi_zero_shot.json")
evaluate_responses("microsoft/Phi-3.5-mini-instruct", "cot", "phi_cot.json")
evaluate_responses("microsoft/Phi-3.5-mini-instruct", "react", "phi_react.json")

Evaluating microsoft/Phi-3.5-mini-instruct with zero_shot:   3%|▎         | 1/30 [00:00<00:03,  7.67it/s]

Error with model microsoft/Phi-3.5-mini-instruct: Error code: 400 - {"message": "Unable to access model microsoft/Phi-3.5-mini-instruct. ", "type_": "invalid_request_error", "code": "model_not_available"}
Model response: None
Error with model microsoft/Phi-3.5-mini-instruct: Error code: 400 - {"message": "Unable to access model microsoft/Phi-3.5-mini-instruct. ", "type_": "invalid_request_error", "code": "model_not_available"}
Model response: None


Evaluating microsoft/Phi-3.5-mini-instruct with zero_shot:  10%|█         | 3/30 [00:02<00:26,  1.02it/s]

Error with model microsoft/Phi-3.5-mini-instruct: Error code: 400 - {"message": "Unable to access model microsoft/Phi-3.5-mini-instruct. ", "type_": "invalid_request_error", "code": "model_not_available"}
Model response: None
Error with model microsoft/Phi-3.5-mini-instruct: Error code: 400 - {"message": "Unable to access model microsoft/Phi-3.5-mini-instruct. ", "type_": "invalid_request_error", "code": "model_not_available"}
Model response: None


Evaluating microsoft/Phi-3.5-mini-instruct with zero_shot:  17%|█▋        | 5/30 [00:05<00:31,  1.27s/it]

...

Error with model microsoft/Phi-3.5-mini-instruct: Error code: 400 - {"message": "Unable to access model microsoft/Phi-3.5-mini-instruct. ", "type_": "invalid_request_error", "code": "model_not_available"}
Model response: None
Error with model microsoft/Phi-3.5-mini-instruct: Error code: 400 - {"message": "Unable to access model microsoft/Phi-3.5-mini-instruct. ", "type_": "invalid_request_error", "code": "model_not_available"}
Model response: None


Evaluating microsoft/Phi-3.5-mini-instruct with zero_shot: 100%|██████████| 30/30 [00:39<00:00,  1.33s/it]


Error with model microsoft/Phi-3.5-mini-instruct: Error code: 400 - {"message": "Unable to access model microsoft/Phi-3.5-mini-instruct. ", "type_": "invalid_request_error", "code": "model_not_available"}
Model response: None
Error with model microsoft/Phi-3.5-mini-instruct: Error code: 400 - {"message": "Unable to access model microsoft/Phi-3.5-mini-instruct. ", "type_": "invalid_request_error", "code": "model_not_available"}
Model response: None


Evaluating microsoft/Phi-3.5-mini-instruct with cot:   0%|          | 0/30 [00:02<?, ?it/s]


InvalidRequestError: Error code: 400 - {"message": "Unable to access model microsoft/Phi-3.5-mini-instruct. ", "type_": "invalid_request_error", "code": "model_not_available"}